In [16]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [17]:
df = pd.read_csv('fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
## TODO - Add preprocess to pd.DataFrame
def preprocess(df: pd.DataFrame, column_name: str) -> None:
    df[column_name] = df[column_name].apply(lambda x: str(x).lower())
    translation = str.maketrans('','', string.punctuation)
    stemmer = PorterStemmer()
    df[column_name] = df[column_name].apply(lambda sentence: ' '.join([stemmer.stem(word.translate(translation)) \
                                                                       for word in word_tokenize(sentence) \
                                                                       if word.isalpha() and word not in stopwords.words('english')]))
    return(None)
preprocess(df, 'title')

In [19]:
df.head()

,id,title,author,text,label
0,0,hous dem aid even see comey letter jason chaff...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,flynn hillari clinton big woman campu breitbart,Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,truth might get fire,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,civilian kill singl us airstrik identifi,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,iranian woman jail fiction unpublish stori wom...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
df.drop('author', axis = 1, inplace = True)
df.dropna(inplace = True)

In [21]:
X = df.drop('label', axis = 1)
y = df['label']

In [22]:
X.shape

(20761, 3)

In [23]:
y.shape

(20761,)

In [24]:
tf.__version__

'2.3.1'

#### One-Hot Representation

In [25]:
voc_size = 5000

In [26]:
df.head()

,id,title,text,label
0,0,hous dem aid even see comey letter jason chaff...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,flynn hillari clinton big woman campu breitbart,Ever get the feeling your life circles the rou...,0
2,2,truth might get fire,"Why the Truth Might Get You Fired October 29, ...",1
3,3,civilian kill singl us airstrik identifi,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,iranian woman jail fiction unpublish stori wom...,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
onehot_repr = [one_hot(sentence, voc_size) for sentence in df.title]
onehot_repr[:5]

[[3387, 2104, 130, 3976, 4105, 681, 1111, 332, 3678, 2496],
 [3183, 331, 3246, 1176, 3243, 3999, 4973],
 [4155, 3957, 679, 1743],
 [139, 1429, 3848, 1496, 3647, 3356],
 [2172, 3243, 846, 1548, 2945, 2506, 3243, 941, 4529, 2829]]

#### Embedding Representation

In [28]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ...  332 3678 2496]
 [   0    0    0 ... 3243 3999 4973]
 [   0    0    0 ... 3957  679 1743]
 ...
 [   0    0    0 ...  212 3346 1615]
 [   0    0    0 ... 3011 2992 4004]
 [   0    0    0 ...    0 3883 1178]]


In [29]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3387,
       2104,  130, 3976, 4105,  681, 1111,  332, 3678, 2496], dtype=int32)

In [30]:
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [31]:
np.array(embedded_docs).shape, y.shape

((20761, 20), (20761,))

In [32]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
X_final[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3387,
       2104,  130, 3976, 4105,  681, 1111,  332, 3678, 2496], dtype=int32)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### Model Training

In [35]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
218/218 [==============================] - 8s 35ms/step - loss: 0.2663 - accuracy: 0.8822 - val_loss: 0.1875 - val_accuracy: 0.9225
Epoch 2/10
218/218 [==============================] - 7s 32ms/step - loss: 0.1387 - accuracy: 0.9484 - val_loss: 0.2033 - val_accuracy: 0.9240
Epoch 3/10
218/218 [==============================] - 8s 36ms/step - loss: 0.1020 - accuracy: 0.9623 - val_loss: 0.2067 - val_accuracy: 0.9219
Epoch 4/10
218/218 [==============================] - 8s 35ms/step - loss: 0.0800 - accuracy: 0.9707 - val_loss: 0.2202 - val_accuracy: 0.9218
Epoch 5/10
218/218 [==============================] - 8s 35ms/step - loss: 0.0625 - accuracy: 0.9765 - val_loss: 0.2429 - val_accuracy: 0.9189
Epoch 6/10
218/218 [==============================] - 8s 35ms/step - loss: 0.0440 - accuracy: 0.9853 - val_loss: 0.2779 - val_accuracy: 0.9228
Epoch 7/10
218/218 [==============================] - 8s 35ms/step - loss: 0.0292 - accuracy: 0.9905 - val_loss: 0.3382 - val_accuracy: 0.9124

#### Adding Dropout

In [36]:
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
218/218 [==============================] - 8s 38ms/step - loss: 0.2844 - accuracy: 0.8773 - val_loss: 0.1897 - val_accuracy: 0.9244
Epoch 2/10
218/218 [==============================] - 8s 37ms/step - loss: 0.1395 - accuracy: 0.9475 - val_loss: 0.1807 - val_accuracy: 0.9253
Epoch 3/10
218/218 [==============================] - 9s 40ms/step - loss: 0.1055 - accuracy: 0.9626 - val_loss: 0.1886 - val_accuracy: 0.9260
Epoch 4/10
218/218 [==============================] - 11s 51ms/step - loss: 0.0843 - accuracy: 0.9700 - val_loss: 0.2282 - val_accuracy: 0.9219
Epoch 5/10
218/218 [==============================] - 10s 44ms/step - loss: 0.0606 - accuracy: 0.9778 - val_loss: 0.2449 - val_accuracy: 0.9206
Epoch 6/10
218/218 [==============================] - 10s 45ms/step - loss: 0.0413 - accuracy: 0.9861 - val_loss: 0.3217 - val_accuracy: 0.9162
Epoch 7/10
218/218 [==============================] - 8s 38ms/step - loss: 0.0266 - accuracy: 0.9920 - val_loss: 0.3500 - val_accuracy: 0.9

#### Performance Metrics and Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y_pred = model.predict_classes(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)